# Multi-agent Collaboration for Financial Analysis

Agents collaborate with each other.

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
import getpass

# Prompt the user for input (hidden)
api_key = getpass.getpass("Enter the openai api key to set it as environment variable 'OPENAI_API_KEY': ")
api_base = getpass.getpass("Enter the openai api base to set it as environment variable 'OPENAI_API_BASE': ")
# serper_api_key = getpass.getpass("Enter the SERPER_API_KEY for efficient search: ")
os.environ["SERPER_API_KEY"] = getpass.getpass("Enter the serper api key to set it as environment variable 'SERPER_API_KEY': ")

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## crewAI Tools

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/opt/homebrew/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/homebrew/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/opt/homebrew/lib/python3.10/site-packa

## Creating Agents

In [4]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [5]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [8]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [9]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [10]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [11]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [12]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [14]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2254b8ab-e9fd-491d-91b3-97298196a0ce                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4c543f2e-9c1f-45d1-bb5c-5efe1e19327d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4c543f2e-9c1f-45d1-bb5c-5efe1e19327d
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


🚀 Crew: crew
└── 📋 Task: 4c543f2e-9c1f-45d1-bb5c-5efe1e19327d
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress
        └── 🧠 Thinking...

...

 Error during LLM call: litellm.ContextWindowExceededError: litellm.BadRequestError: ContextWindowExceededError: OpenAIException - Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 20060 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
 An unknown error occurred. Please check the details below.
 Error details: litellm.ContextWindowExceededError: litellm.BadRequestError: ContextWindowExceededError: OpenAIException - Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 20060 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


🚀 Crew: crew
├── 📋 Task: 4c543f2e-9c1f-45d1-bb5c-5efe1e19327d
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   ├── 🔧 Used Search the internet with Serper (1)
│   │   ├── 🔧 Used Read website content (1)
│   │   ├── 🔧 Using Delegate work to coworker (3)
│   │   └── 🔧 Using Ask the Data Analyst about their progress and if they need any additional resources or 
│   │       information to complete the task effectively. (1)
│   ├── 🤖 Agent: Data Analyst
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (2)
│   │   ├── 🔧 Used Read website content (2)
│   │   ├── 🔧 Using Delegate work to coworker (15)
│   │   └── 🔧 Using Ask question to coworker (1)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (3)
│       └── 🔧 Using Ask question to coworker (6)
├── 📋 Task: ebf65bae-429d-4aa4-b0ee-91e9da83d4ec
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Read website content (5)
│       └── 🔧 Used Search the internet with Serper (4)
├── 📋 Task: 22706a1f-e29b-4d0f-9b3b-ad3a18836464
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   ├── 🔧 Used Read website content (6)
│   │   └── 🔧 Using Delegate work to coworker (16)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (5)
│       └── 🔧 Used Read website content (7)
└── 📋 Task: 1179ffdd-f9f9-47f7-be81-248800c8d871
       Assigned to: Crew Manager
       Status: ❌ Failed
    └── 🤖 Agent: Crew Manager
            Status: In Progress
        ├── 🔧 Used Read website content (10)
        ├── 🔧 Used Search the internet with Serper (6)
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 1179ffdd-f9f9-47f7-be81-248800c8d871                                                                     │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 2254b8ab-e9fd-491d-91b3-97298196a0ce                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ContextWindowExceededError: litellm.ContextWindowExceededError: litellm.BadRequestError: ContextWindowExceededError: OpenAIException - Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 20060 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

NameError: name 'result' is not defined